In [43]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# working model for tensorfusion

In [44]:
# 필요 라이브러리

import pickle
import torch
from torchmetrics import F1Score
import pandas as pd
import numpy as np
from glob import glob
from collections import Counter
import os
import pandas as pd
from datasets import Dataset
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import random


print(torch.__version__)

1.13.1+cu117


In [45]:
# Hyperparameters
BATCH_SIZE = 256
EPOCHS = 15
LR = 1e-2

In [46]:
ROOT = "/home/arplab/project/paradeigma/multi_modal/"

In [47]:
import pickle
# Annotation, embedded data 불러오기
with open(ROOT + 'model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl', 'rb') as f:
    annotation_20_nonmissing = pickle.load(f)

with open(ROOT + 'model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl', 'rb') as f:
    annotation_19_nonmissing = pickle.load(f)
    
with open(ROOT + 'model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl', 'rb') as f:
    embedding_20_dataset = pickle.load(f)

with open(ROOT + 'model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl', 'rb') as f:
    embedding_19_dataset = pickle.load(f)    

In [48]:
# 19년도 20년도 embedding 병합

wav_19_em = []
for embedding in embedding_19_dataset[0].values():
    wav_19_em.append(embedding)
wav_19_em = torch.cat(wav_19_em) 
    
txt_19_em = []
for embedding in embedding_19_dataset[1].values():
    txt_19_em.append(embedding)
txt_19_em = torch.cat(txt_19_em) 

wav_20_em = []
for embedding in embedding_20_dataset[0].values():
    wav_20_em.append(embedding)
wav_20_em = torch.cat(wav_20_em) 
    
txt_20_em = []
for embedding in embedding_20_dataset[1].values():
    txt_20_em.append(embedding)
txt_20_em = torch.cat(txt_20_em) 


In [49]:
wav_19_em.shape

torch.Size([9008, 149, 1024])

In [50]:
# 19년도 nonneutral data embedding
nonneutral_19_index = pd.Series(annotation_19_nonmissing[annotation_19_nonmissing['Emotion'] != 'neutral'].index)
nonneutral_19_em_wav = []
nonneutral_19_em_txt = []
for index in nonneutral_19_index:
    nonneutral_19_em_wav.append(wav_20_em[index].view(1, 149, 1024))
    nonneutral_19_em_txt.append(txt_20_em[index].view(1, 80, 768))
    
nonneutral_19_em_wav = torch.concat(nonneutral_19_em_wav, dim=0)
nonneutral_19_em_txt = torch.concat(nonneutral_19_em_txt, dim=0)

In [51]:
session_20_list = list(embedding_20_dataset[0].keys())
test_val = random.sample(session_20_list, k=16)
test_session = sorted(test_val[:8])
val_session = sorted(test_val[8:])
train_session = sorted(list(set(session_20_list) - (set(test_session) | set(val_session))))
train_session

['Sess01',
 'Sess02',
 'Sess03',
 'Sess04',
 'Sess08',
 'Sess09',
 'Sess10',
 'Sess13',
 'Sess14',
 'Sess15',
 'Sess16',
 'Sess18',
 'Sess19',
 'Sess21',
 'Sess22',
 'Sess23',
 'Sess26',
 'Sess33',
 'Sess34',
 'Sess36',
 'Sess37',
 'Sess38']

In [52]:
def session_check(segment_id, session):
    if session in segment_id:
        return True
    else : return False

train_df = pd.DataFrame()
train_embedding_wav = []
train_embedding_txt = []
for session in train_session:    
    df = annotation_20_nonmissing[annotation_20_nonmissing['Segment ID'].apply(session_check, session=session)]
    train_df = train_df.append(df, ignore_index=True)
    train_embedding_wav.append(embedding_20_dataset[0][session])
    train_embedding_txt.append(embedding_20_dataset[1][session])
train_embedding_wav = torch.concat(train_embedding_wav)
train_embedding_txt = torch.concat(train_embedding_txt)
    
val_df = pd.DataFrame()
val_embedding_wav = []
val_embedding_txt = []
for session in val_session:    
    df = annotation_20_nonmissing[annotation_20_nonmissing['Segment ID'].apply(session_check, session=session)]
    val_df = val_df.append(df, ignore_index=True)
    val_embedding_wav.append(embedding_20_dataset[0][session])
    val_embedding_txt.append(embedding_20_dataset[1][session])
val_embedding_wav = torch.concat(val_embedding_wav)
val_embedding_txt = torch.concat(val_embedding_txt)
    
test_df = pd.DataFrame()
test_embedding_wav = []
test_embedding_txt = []
for session in test_session:    
    df = annotation_20_nonmissing[annotation_20_nonmissing['Segment ID'].apply(session_check, session=session)]
    test_df = test_df.append(df, ignore_index=True)
    test_embedding_wav.append(embedding_20_dataset[0][session])
    test_embedding_txt.append(embedding_20_dataset[1][session])
test_embedding_wav = torch.concat(test_embedding_wav)
test_embedding_txt = torch.concat(test_embedding_txt)    


/tmp/ipykernel_219623/3324316.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
/tmp/ipykernel_219623/3324316.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val_df = val_df.append(df, ignore_index=True)
/tmp/ipykernel_219623/3324316.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(df, ignore_index=True)


In [53]:
train_df = train_df.append(annotation_19_nonmissing.loc[nonneutral_19_index], ignore_index=True)

train_embedding_wav = torch.concat([train_embedding_wav, nonneutral_19_em_wav])
train_embedding_txt = torch.concat([train_embedding_txt, nonneutral_19_em_txt])

print('Train',len(train_df), train_embedding_wav.shape, train_embedding_txt.shape)
print('Val',len(val_df), val_embedding_wav.shape, val_embedding_txt.shape)
print('Test',len(test_df), test_embedding_wav.shape, test_embedding_txt.shape)


/tmp/ipykernel_219623/4029699924.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(annotation_19_nonmissing.loc[nonneutral_19_index], ignore_index=True)


Train 12842 torch.Size([12842, 149, 1024]) torch.Size([12842, 80, 768])
Val 2434 torch.Size([2434, 149, 1024]) torch.Size([2434, 80, 768])
Test 2976 torch.Size([2976, 149, 1024]) torch.Size([2976, 80, 768])


In [54]:
encode_dict = {'angry':0, 'disgust':1, 'fear':2,'happy':3,'neutral':4, 'sad':5, 'surprise':6,  
               'neutral;surprise': 450, 'neutral;sad': 460, 'happy;neutral': 340, 
               'angry;neutral': 40, 'disgust;neutral': 140, 'fear;neutral': 240, 
               'happy;surprise': 350, 'angry;happy;neutral': 7340, 'angry;disgust': 10, 
               'happy;neutral;surprise': 3450, 'fear;happy': 230,'fear;happy;neutral': 2340,
               'angry;disgust;neutral': 7140, 'disgust;neutral;sad': 1460, 
               'happy;sad': 360, 'disgust;happy;neutral': 3410, 'angry;fear': 20, 'angry;fear;neutral':7240,
               'angry;fear;surprise': 7250, 'angry;happy': 730, 'angry;neutral;surprise':7450, 
               'angry;sad': 60, 'angry;surprise': 50, 'disgust;fear':120, 'disgust;happy': 130,
               'disgust;neutral;surprise':1450, 'disgust;sad': 160, 'disgust;surprise':150, 
               'fear;neutral;surprise':2450, 'fear;sad': 260, 'fear;surprise':250, 'happy;neutral;sad':3460,
               'neutral;sad;surprise':4560, 'sad;surprise': 560,
               'angry;disgust;fear;neutral;sad': 10000,'angry;disgust;fear;neutral;surprise':20000}

In [55]:
encode_dict = {k: v for k, v in sorted(encode_dict.items(), key=lambda item: item[1])}
decode_dict = {v: k for k, v in sorted(encode_dict.items(), key=lambda item: item[1])}
print(encode_dict)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6, 'angry;disgust': 10, 'angry;fear': 20, 'angry;neutral': 40, 'angry;surprise': 50, 'angry;sad': 60, 'disgust;fear': 120, 'disgust;happy': 130, 'disgust;neutral': 140, 'disgust;surprise': 150, 'disgust;sad': 160, 'fear;happy': 230, 'fear;neutral': 240, 'fear;surprise': 250, 'fear;sad': 260, 'happy;neutral': 340, 'happy;surprise': 350, 'happy;sad': 360, 'neutral;surprise': 450, 'neutral;sad': 460, 'sad;surprise': 560, 'angry;happy': 730, 'disgust;neutral;surprise': 1450, 'disgust;neutral;sad': 1460, 'fear;happy;neutral': 2340, 'fear;neutral;surprise': 2450, 'disgust;happy;neutral': 3410, 'happy;neutral;surprise': 3450, 'happy;neutral;sad': 3460, 'neutral;sad;surprise': 4560, 'angry;disgust;neutral': 7140, 'angry;fear;neutral': 7240, 'angry;fear;surprise': 7250, 'angry;happy;neutral': 7340, 'angry;neutral;surprise': 7450, 'angry;disgust;fear;neutral;sad': 10000, 'angry;disgust;fear;neutral;surprise': 2

In [56]:
train_df['Emotion'] = train_df['Emotion'].apply(lambda x: encode_dict[x])
val_df['Emotion'] = val_df['Emotion'].apply(lambda x: encode_dict[x])
test_df['Emotion'] = test_df['Emotion'].apply(lambda x: encode_dict[x])

In [57]:
# Padding 함수
def sequence_padding(ts_list, padding_length = 50, mode = 'constant'):
    
    padding_value=0
    
    if (type(ts_list) != type([])) :
        ts_list = [padding_value] * padding_length
    
    elif len(ts_list) >= padding_length :
        ts_list = ts_list[0:padding_length]
    
    elif mode == 'constant':
        length = padding_length - len(ts_list)
        extend_list = [padding_value] * length
        ts_list = ts_list + extend_list    

    elif mode == 'replicate':
        
        quotient = padding_length // len(ts_list)
        remainder = padding_length % len(ts_list)
        result = ts_list * quotient
        result += ts_list[:remainder]
        ts_list = result

    return ts_list            

In [58]:
train_df

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP,Scaled EDA length,Scaled TEMP length
0,Sess01_script01_User002M_001,4,3.4,2.9,"[0, 0, 0, 0, 10, 0, 0]","[0, 0, 6, 4, 0]","[0, 1, 9, 0, 0]","[-0.06791500000000017, -0.110201, -0.025627999...","[34.81, 34.81, 34.81, 34.79, 34.79, 34.79, 34....",32,32,"[-0.3956120608509403, -0.6460741139849967, -0....","[-0.5852354223355396, -0.5852354223355396, -0....",31,32
1,Sess01_script01_User002M_002,4,3.1,2.9,"[0, 0, 0, 0, 10, 0, 0]","[0, 0, 9, 1, 0]","[0, 1, 9, 0, 0]","[0.006407000000000274, -0.021784000000000248, ...","[34.79, 34.79, 34.79, 34.77, 34.77, 34.77, 34....",47,47,"[0.04460079241088465, -0.12237588402818134, -0...","[-0.7735061037287296, -0.7735061037287296, -0....",46,47
2,Sess01_script01_User002M_003,4,3.1,3.0,"[0, 0, 0, 0, 10, 0, 0]","[0, 0, 9, 1, 0]","[0, 0, 10, 0, 0]","[0.07944700000000005, 0.05125599999999997, 0.0...","[34.75, 34.75, 34.75, 34.75, 34.79, 34.79, 34....",33,33,"[0.4772202967601137, 0.31024362032105035, 0.27...","[-1.1500474665150429, -1.1500474665150429, -1....",32,33
3,Sess01_script01_User002M_004,4,3.7,3.1,"[0, 0, 0, 1, 9, 0, 0]","[0, 0, 3, 7, 0]","[0, 0, 9, 1, 0]","[0.0038439999999999586, -0.03331699999999982, ...","[34.77, 34.77, 34.77, 34.77, 34.77, 34.77, 34....",45,45,"[0.029420017634772744, -0.19068640899613712, -...","[-0.9617767851218528, -0.9617767851218528, -0....",44,45
4,Sess01_script01_User001F_001,4,3.8,2.8,"[0, 0, 0, 4, 6, 0, 0]","[0, 0, 3, 6, 1]","[0, 2, 8, 0, 0]","[0.1241910000000006, 0.09474499999999964, 0.03...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....",9,9,"[1.8771197424131327, 1.4195480026451441, 0.544...","[-2.4621880639731812, -2.4621880639731812, -2....",8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12837,Sess20_impro04_M018,5,2.3,2.1,"[0, 0, 0, 0, 3, 7, 0]","[2, 4, 3, 1, 0]","[0, 9, 1, 0, 0]","[0.550024, 0.547461, 0.54618, 0.543617, 0.5461...","[31.27, 31.27, 31.27, 31.27, 31.27, 31.27, 31....",31,31,"[-1.0596542781342695, -1.091127738705599, -1.1...","[-0.12418082175855716, -0.12418082175855716, -...",31,31
12838,Sess20_impro04_F023,5,2.5,2.3,"[0, 0, 0, 0, 2, 8, 0]","[0, 6, 3, 1, 0]","[1, 5, 4, 0, 0]","[0.216838, 0.220682, 0.221964, 0.223245, 0.221...","[29.77, 29.77, 29.77, 29.77, 29.77, 29.77, 29....",46,46,"[-0.8593367498709915, -0.7994211382956963, -0....","[-1.3249127202479716, -1.3249127202479716, -1....",46,46
12839,Sess20_impro04_M019,5,2.1,1.8,"[0, 0, 0, 0, 1, 9, 0]","[4, 2, 3, 1, 0]","[3, 6, 1, 0, 0]","[0.519269, 0.517987, 0.519269, 0.517987, 0.516...","[31.31, 31.31, 31.31, 31.31, 31.29, 31.29, 31....",56,56,"[-1.437323525060235, -1.4530663953108964, -1.4...","[0.697453930425023, 0.697453930425023, 0.69745...",56,56
12840,Sess20_impro04_F024,3,3.9,3.2,"[0, 0, 0, 7, 3, 0, 0]","[0, 0, 1, 9, 0]","[0, 1, 6, 3, 0]","[0.207868, 0.207868, 0.210431, 0.211712, 0.212...","[29.73, 29.73, 29.73, 29.73, 29.73, 29.73, 29....",52,52,"[-0.9991502347384216, -0.9991502347384216, -0....","[-1.6382728540476073, -1.6382728540476073, -1....",52,52


In [59]:
# 시계열 데이터 2020 + 2019
train_df['Scaled EDA'] = train_df['Scaled EDA'].apply(sequence_padding)
train_df['Scaled TEMP'] = train_df['Scaled TEMP'].apply(sequence_padding)

val_df['Scaled EDA'] = val_df['Scaled EDA'].apply(sequence_padding)
val_df['Scaled TEMP'] = val_df['Scaled TEMP'].apply(sequence_padding)

test_df['Scaled EDA'] = test_df['Scaled EDA'].apply(sequence_padding)
test_df['Scaled TEMP'] = test_df['Scaled TEMP'].apply(sequence_padding)

In [60]:
# https://stats.stackexchange.com/questions/107874/how-to-deal-with-a-skewed-class-in-binary-classification-having-many-features


In [61]:
torch.set_default_dtype(torch.float32)

# torch dataset 만들기
- 참고: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [62]:
class EtriDataset(Dataset):
    def __init__(self, file_names, 
                 text_embeddings, 
                 wav_embeddings, 
                 Temp,
                 EDA,
                 Emotion,
                 Emotion_ext, 
                 Arousal, 
                 Valence):
        self.file_names = file_names
        self.text_embeddings = text_embeddings
        self.wav_embeddings = wav_embeddings
        self.temp = Temp
        self.eda = EDA
        self.label_emotion = Emotion
        self.label_emotion_ext = Emotion_ext
        self.label_arousal = Arousal
        self.label_valence = Valence
        
    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        text_embeddings = self.text_embeddings[idx]
        wav_embeddings = self.wav_embeddings[idx]
        temp = self.temp[idx]
        eda = self.eda[idx]
        label_emotion = self.label_emotion[idx]
        label_emotion_ext = self.label_emotion_ext[idx]
        label_arousal = self.label_arousal[idx]
        label_valence = self.label_valence[idx]
        return text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence

In [63]:
train_df.columns

Index(['Segment ID', 'Emotion', 'Valence', 'Arousal', 'emotion_vector',
       'valence_vector', 'arousal_vector', 'EDA', 'TEMP', 'EDA length',
       'TEMP length', 'Scaled EDA', 'Scaled TEMP', 'Scaled EDA length',
       'Scaled TEMP length'],
      dtype='object')

In [64]:
test_embedding_txt

tensor([[[-5.4479e-01, -3.1275e-02, -1.0944e+00,  ..., -6.5808e-01,
           1.1027e+00, -2.9669e-01],
         [-6.1710e-01, -4.8289e-03, -1.0378e+00,  ..., -6.2813e-01,
           1.1145e+00, -2.7613e-01],
         [-5.9215e-01, -9.6105e-03, -1.1005e+00,  ..., -6.2060e-01,
           1.1014e+00, -2.8010e-01],
         ...,
         [-5.5526e-01, -4.4838e-04, -1.1606e+00,  ..., -6.1127e-01,
           9.6024e-01, -3.0464e-01],
         [-5.5526e-01, -4.4838e-04, -1.1606e+00,  ..., -6.1127e-01,
           9.6024e-01, -3.0464e-01],
         [-5.5526e-01, -4.4838e-04, -1.1606e+00,  ..., -6.1127e-01,
           9.6024e-01, -3.0464e-01]],

        [[-5.2497e-01, -3.6781e-02, -1.1262e+00,  ..., -6.9652e-01,
           1.0444e+00, -2.8279e-01],
         [-6.1824e-01,  2.6758e-03, -1.1191e+00,  ..., -7.0198e-01,
           1.0386e+00, -2.3687e-01],
         [-5.8200e-01, -1.2159e-02, -1.0670e+00,  ..., -6.5854e-01,
           1.0871e+00, -2.3476e-01],
         ...,
         [-5.3314e-01,  2

In [65]:
# data load 및 나누기: https://076923.github.io/posts/Python-pytorch-11/

# session을 통합시킨 데이터 셋을 만들었을 때
train_dataset = EtriDataset(file_names = train_df['Segment ID'],
                      text_embeddings = train_embedding_txt,
                      wav_embeddings = train_embedding_wav,
                      Emotion = train_df['Emotion'],
                      Arousal = train_df['Arousal'],
                      Valence = train_df['Valence'],
                      EDA = torch.concat(list(train_df['Scaled EDA'].apply(lambda x: torch.tensor(x).view(1,-1)))), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Temp = torch.concat(list(train_df['Scaled TEMP'].apply(lambda x: torch.tensor(x).view(1,-1)))), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Emotion_ext = torch.concat(list(train_df['emotion_vector'].apply(lambda x: torch.tensor(x).view(1,-1))))) ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함

val_dataset = EtriDataset(file_names = val_df['Segment ID'],
                      text_embeddings = val_embedding_txt,
                      wav_embeddings = val_embedding_wav,
                      Emotion = val_df['Emotion'],
                      Arousal = val_df['Arousal'],
                      Valence = val_df['Valence'],
                      EDA = torch.concat(list(val_df['Scaled EDA'].apply(lambda x: torch.tensor(x).view(1,-1)))), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Temp = torch.concat(list(val_df['Scaled TEMP'].apply(lambda x: torch.tensor(x).view(1,-1)))), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Emotion_ext = torch.concat(list(val_df['emotion_vector'].apply(lambda x: torch.tensor(x).view(1,-1))))) ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함

test_dataset = EtriDataset(file_names = test_df['Segment ID'],
                      text_embeddings = test_embedding_txt,
                      wav_embeddings = test_embedding_txt,
                      Emotion = test_df['Emotion'],
                      Arousal = test_df['Arousal'],
                      Valence = test_df['Valence'],
                      EDA = torch.concat(list(test_df['Scaled EDA'].apply(lambda x: torch.tensor(x).view(1,-1)))), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Temp = torch.concat(list(test_df['Scaled TEMP'].apply(lambda x: torch.tensor(x).view(1,-1)))), ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함
                      Emotion_ext = torch.concat(list(test_df['emotion_vector'].apply(lambda x: torch.tensor(x).view(1,-1))))) ## 일부 세션 데이터만 사용하므로, 길이를 맞춰주기 위해 일부 slicing함


In [66]:
train_size = len(train_dataset)
validation_size = len(val_dataset)
test_size = len(test_dataset)

print(train_size, test_size, validation_size)
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(val_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

12842 2976 2434
Training Data Size : 12842
Validation Data Size : 2434
Testing Data Size : 2976


In [67]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# NetWork 만들기

In [68]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)
# 위의 오류가 해결되기 전까진 일단 cpu를 가지고 모델을 돌리기로 한다. 
# device = 'cpu'

Using cuda device


In [69]:
class MLPNetwork_pre(nn.Module):
    def __init__(self, input_length, input_width):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width, 768)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(768)
        self.fc2 = nn.Linear(768, 512)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 32)
        self.gelu3 = nn.GELU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        output = self.gelu3(x)
        return output
    
class ConvNetwork_pre(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels = 1, out_channels= 32, kernel_size = 16)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 16)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        x = torch.transpose(x, 1,2)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        output = self.relu2(x)
        return output    
    
class ConvNetwork_middle(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=1, kernel_size=11)
        self.relu1 = nn.ReLU()
    
    def forward(self, x):
        x = self.conv1(x)
        output = self.relu1(x)    
        return output.squeeze()
    
class ConvNetwork_final(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.conv2d_1 = nn.Conv2d(in_channels = 10, out_channels = 64, kernel_size=2)
        self.leakyrelu_1 = nn.LeakyReLU()
        self.maxpool2d_1 = nn.MaxPool2d(2)
        self.conv2d_2 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size=2)
        self.leakyrelu_2 = nn.LeakyReLU()
        self.maxpool2d_2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1568, 64)
        self.leakyrelu_3 = nn.LeakyReLU()
        self.batchnorm = nn.BatchNorm1d(64)
        self.drop = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.leakyrelu_1(x)
        x = self.maxpool2d_1(x)
        x = self.conv2d_2(x)
        x = self.leakyrelu_2(x)
        x = self.maxpool2d_2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.leakyrelu_3(x)
        x = self.batchnorm(x)
        x = self.drop(x)
        output = self.fc2(x)  
        return output
        



In [70]:
class TensorFusionMixer(nn.Module):
    def __init__(self, ModelA, ModelB, ModelC, ModelD, ModelE, ModelF):
        super().__init__()
        self.ModelA = ModelA
        self.ModelB = ModelB
        self.ModelC = ModelC
        self.ModelD = ModelD
        self.ModelE = ModelE
        self.Model_cnn_final = ModelF
        # self.softmax = nn.Softmax(dim=1)
        
    def tensor_fusion(self, batch_arr1, batch_arr2, batch_arr3):
        fusion_matrix_lst = []
        for i, (arr1, arr2, arr3) in enumerate(zip(batch_arr1, batch_arr2, batch_arr3)):
            # arr1 = arr1.unsqueeze(-1).unsqueeze(-1)
            # arr2 = arr2.unsqueeze(0).unsqueeze(-1)
            # arr3 = arr3.squeeze().unsqueeze(0).unsqueeze(0)
            
            arr1 = arr1.unsqueeze(0).unsqueeze(0)
            arr2 = arr2.unsqueeze(0).unsqueeze(-1)
            arr3 = arr3.unsqueeze(-1).unsqueeze(-1)
            
            # outer_matrix = torch.einsum('i,j,kp->ijk', arr1, arr2, arr3)
            kron_matrix = torch.kron(arr3, torch.kron(arr2, arr1,))
            l, w, d = kron_matrix.shape
            
            kron_matrix = kron_matrix.view(-1, l, w, d)
            fusion_matrix_lst.append(kron_matrix)
            
        fusion_matrix = torch.concat(fusion_matrix_lst)
        # fusion_matrix = fusion_matrix.unsqueeze(-1)
        
        return fusion_matrix
        
    def forward(self, x1, x2, x3, x4):
        x1 = self.ModelA(x1)
        x2 = self.ModelB(x2)
        x3 = self.ModelC(x3)
        x4 = self.ModelD(x4)
        x5 = torch.concat([x3, x4], dim=1)
        x5 = self.ModelE(x5)
        fusion_matrix = self.tensor_fusion(x1, x2, x5)
        # x5 = torch.cat([x3,x4], dim=0)
        output = self.Model_cnn_final(fusion_matrix) 
        return output




In [71]:
# txt_input_length, txt_input_width = raw_dataset[session]['text_embeddings'][0].shape | 마지막엔 지울 것
# _, wav_input_length, wav_input_width = raw_dataset[session]['wav_embeddings'][0].shape
txt_input_length, txt_input_width = train_dataset.text_embeddings[0].shape
wav_input_length, wav_input_width = train_dataset.wav_embeddings[0].shape
temp_input_length = train_dataset.temp[0].shape[0]
eda_input_length = train_dataset.eda[0].shape[0]

# tf_mixer에 들어갈 wav mlp, txt mlp 선언
model_mlp_txt = MLPNetwork_pre(txt_input_length,txt_input_width).to(device)
model_mlp_wav = MLPNetwork_pre(wav_input_length,wav_input_width).to(device)
# model_conv_temp = ConvNetwork_pre(temp_input_length).to(device)
model_conv_temp = ConvNetwork_pre().to(device)
model_conv_eda = ConvNetwork_pre().to(device)
# model_conv_eda = ConvNetwork_pre(eda_input_length).to(device)
model_conv_middle = ConvNetwork_middle().to(device)
# model_cnn_final = ConvNetwork_final(32).to(device)
model_cnn_final = ConvNetwork_final().to(device)
# 최종 모델 선언
model_tf_cnn_mixer = TensorFusionMixer(ModelA = model_mlp_txt, 
                                   ModelB = model_mlp_wav,
                                   ModelC = model_conv_temp,
                                   ModelD = model_conv_eda,
                                   ModelE = model_conv_middle,
                                   ModelF = model_cnn_final
                                   ).to(device)

# model 병렬 학습 처리
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_mlp_txt = nn.DataParallel(model_mlp_txt).to(device)
    model_mlp_wav = nn.DataParallel(model_mlp_wav).to(device)
    model_conv_temp = nn.DataParallel(model_conv_temp).to(device)
    model_conv_eda = nn.DataParallel(model_conv_eda).to(device)
    model_tf_cnn_mixer = nn.DataParallel(model_tf_cnn_mixer).to(device)
    
print(model_tf_cnn_mixer)

Let's use 4 GPUs!
DataParallel(
  (module): TensorFusionMixer(
    (ModelA): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=61440, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=True)
      (gelu2): GELU(approximate='none')
      (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc3): Linear(in_features=512, out_features=32, bias=True)
      (gelu3): GELU(approximate='none')
    )
    (ModelB): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=152576, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=T

In [72]:
class CCC(nn.Module):
    def __init__(self):
        super(CCC, self).__init__()
        self.mean = torch.mean
        self.var = torch.var
        self.sum = torch.sum
        self.sqrt = torch.sqrt
        self.std = torch.std
        
    def forward(self, pred, target):
        mean_gt = self.mean (target, 0)
        mean_pred = self.mean (pred, 0)
        var_gt = self.var (target, 0)
        var_pred = self.var (pred, 0)
        v_pred = pred - mean_pred
        v_gt = target - mean_gt
        cor = self.sum (v_pred * v_gt) / (self.sqrt(self.sum(v_pred ** 2)) * self.sqrt(self.sum(v_gt ** 2)))
        sd_gt = self.std(target)
        sd_pred = self.std(pred)
        numerator = 2 * cor * sd_gt * sd_pred
        denominator = var_gt + var_pred + (mean_gt-mean_pred) ** 2
        ccc = numerator / denominator
        return ccc
    
ccc = CCC()

In [73]:
class CCCLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,inputs,targets):
        self.inputs = inputs
        self.targets = targets
        # pred_v = inputs[:,0]
        # pred_a = inputs[:,1]
        # y_v  = targets[:,0]
        # y_a  = targets[:,1]

        return 1 - ccc(self.inputs, self.targets)
    
cccl = CCCLoss()

# 학습을 위한 train, test method 만들기

In [74]:
def train(dataloader, model, loss_fn, optimizer):
    # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
    return_loss = 0
    for batch, (X_txt, X_wav, X_temp, X_eda, 
                    label_emotion, label_emotion_ext, label_arousal, label_valence) in enumerate(dataloader): 

        # 예측 오류 계산 
        X_txt, X_wav, X_temp, X_eda, y_v= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),label_valence.type(torch.float32).to(device)
        
        X_temp = X_temp.unsqueeze(dim=-1)
        X_eda = X_eda.unsqueeze(dim=-1)
        y_v = y_v.unsqueeze(dim=-1)
        
        pred = model(X_txt, X_wav, X_temp, X_eda)

        loss = loss_fn(pred, y_v)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            ccc_value = ccc(pred, y_v).item()
            loss= loss.item()
            print(ccc_value)
            print(f"loss: {loss}, ccc : {ccc_value}")
        
            return_loss += loss
    
    return return_loss,ccc_value 

In [75]:
def test(dataloader, model, loss_fn, mode = 'test'):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    ccc_value = 0
    
    with torch.no_grad():
        # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
        for batch, (X_txt, X_wav, X_temp, X_eda, 
                    label_emotion, label_emotion_ext, label_arousal, label_valence) in enumerate(dataloader): 

            # 예측 오류 계산 
            X_txt, X_wav, X_temp, X_eda, y_v= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),label_valence.type(torch.float32).to(device)
            
            X_temp = X_temp.unsqueeze(dim=-1)
            X_eda = X_eda.unsqueeze(dim=-1)
            y_v = y_v.unsqueeze(dim=-1)

            pred = model(X_txt, X_wav, X_temp, X_eda)
            
            loss = loss_fn(pred, y_v)
            test_loss += loss
            ccc_value += ccc(pred, y_v).item()
      
    test_loss /= num_batches 
    ccc_value /= num_batches 
    
    if mode == 'test':
        print(f"loss: {test_loss}, ccc : {ccc_value}")
    elif mode == 'val':
        print(f"loss: {test_loss}, ccc : {ccc_value}")
    return test_loss, ccc_value

# 학습시키기

In [76]:
# # 지난 저장한 모델이 있다면
# PATH = './data/test_model.pkl'
# model_tf_mixer = torch.load(PATH)

## start training mlp fusion mixer

In [77]:
import torch.optim as optim

In [78]:
criterion = CCCLoss() 
optimizer = optim.Adam(model_tf_cnn_mixer.parameters(), lr=LR) # regression
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [79]:
wandb.init(
    project="ETRI_kyungho_regression_only_valence",
    name = f"opt : {optimizer.__class__.__name__}, loss : {criterion.__class__.__name__}, LR : {LR}",
    config={
    "dataset": 'KEMDy19, KEMDy20',
    "epochs": EPOCHS,
    "learning_rate": LR,
    "architecture": "TensorFusion",
    "optimizer": optimizer.__class__.__name__,
    "criterion": criterion.__class__.__name__,
    }
)


In [80]:
for epoch in range(EPOCHS):
    print(f"---------------Epoch {epoch+1}----------------")
    print("Training...............")
    
    loss_train,ccc_value_train = train(train_dataloader,
                 model_tf_cnn_mixer, 
                 criterion, 
                 optimizer)
    print("Validation.............")
    loss_val,ccc_value_val = test(validation_dataloader, 
                    model_tf_cnn_mixer, 
                    criterion, 
                    mode='val')

    wandb.log({"train_loss": loss_train, "val_loss": loss_val, 'train_ccc' : ccc_value_train, 'val_ccc' : ccc_value_val})
    scheduler.step(loss_val)
print("Done!")
wandb.finish()

---------------Epoch 1----------------
Training...............


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_219623/4090816372.py", line 44, in forward
    output = self.Model_cnn_final(fusion_matrix)
  File "/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_219623/132462411.py", line 81, in forward
    output = self.fc2(x)
  File "/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
 